In [122]:
from operator import itemgetter
from itertools import groupby, product

In [123]:
from haversine import haversine, Unit

In [124]:
import numpy as np
import pandas as pd
import matplotlib.colors as mcolors

In [125]:
from ipyleaflet import Map, Marker, MarkerCluster, Polyline, Icon
from ipywidgets import HTML

In [126]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

---

In [127]:
def get_centroid(points):
    n_points = len(points)
    return (
        sum(point[0] for point in points) / n_points,
        sum(point[1] for point in points) / n_points,
    )

In [93]:
def get_routes(route_obj, solution):
    routes = list()
    dists = list()
    loads = list()
    for vehicle in range(route_obj['num_vehicles']):
        index = route_obj._routing.Start(vehicle)
        
        route = list()
        load, dist = 0, 0
        while not route_obj._routing.IsEnd(index):
            node_idx = route_obj._manager.IndexToNode(index)
            route.append(node_idx)
            load += route_obj['demands'][node_idx]
            previous_index = index
            index = solution.Value(route_obj._routing.NextVar(index))
            dist += route_obj._routing.GetArcCostForVehicle(
                previous_index, index,
                vehicle
            )
        route.append(route_obj._manager.IndexToNode(index))
        loads.append(load)
        dists.append(dist)
        
        routes.append(route)
    return routes, loads, dists

In [94]:
class Route:
    def __init__(self, positions, n_vehicles=1, veh_capacity=100, max_travel_dist=3000, span_coef=100):
        self._n_vehicles = n_vehicles
        self.max_travel_dist = max_travel_dist
        self.vehicle_capacity = veh_capacity
        self.span_coef = span_coef
        
        self._depot = (None, None)
        self._drops = list()
        self._boxes = list()
        
        for position in positions:
            if not position['is_drop']:
                continue
            self._drops.append((
                position['latitude'],
                position['longitude']
            ))
            
            self._depot = position['depot_latitude'], position['depot_longitude']
            self._boxes.append(round(position['AverageBoxes']) * 1.0)
            
        self._drops = [self._depot] + self._drops
        self._boxes = [0.0] + self._boxes
        self._n_drops = len(self._drops)
        self._capacities = [veh_capacity] * self._n_vehicles
                
        self._dist_mx = np.zeros((self._n_drops, self._n_drops))
        for i, drop_u in enumerate(self._drops):
            for j, drop_v in enumerate(self._drops):
                self._dist_mx[i, j] = round(haversine(drop_u, drop_v, unit=Unit.METERS))
                
        self.init_or_entities()
                
                
    def init_or_entities(self):
        self._manager = pywrapcp.RoutingIndexManager(
            self._n_drops,
            self['num_vehicles'],
            self['depot']
        )
        self._routing = pywrapcp.RoutingModel(self._manager)
        self._transit_callback_index = self._routing.RegisterTransitCallback(self.distance_callback)
        self._routing.SetArcCostEvaluatorOfAllVehicles(self._transit_callback_index)
        
        self._demand_callback_index = self._routing.RegisterUnaryTransitCallback(self.demand_callback)
        self._routing.AddDimension(
            self._demand_callback_index,
            0,
            self.vehicle_capacity,
            True,
            'Capacity'
        )
        
        self._search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        self._search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        self._search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        self._search_parameters.time_limit.FromSeconds(100)
        
    def solve_or(self):
        return self._routing.SolveWithParameters(self._search_parameters)
                
    def distance_callback(self, from_index, to_index):
        from_node = self._manager.IndexToNode(from_index)
        to_node = self._manager.IndexToNode(to_index)
        return self['distance_matrix'][from_node][to_node]
    
    def demand_callback(self, from_index):
        from_node = self._manager.IndexToNode(from_index)
        return self['demands'][from_node]
        
    def __getitem__(self, label):
        if label == 'distance_matrix':
            return self._dist_mx.tolist()
        if label == 'num_vehicles':
            return self._n_vehicles
        if label == 'depot':
            return 0
        if label == 'num_locations':
            return self._n_drops
        if label == 'demands':
            return self._boxes
        if label == 'vehicle_capacities':
            return self._capacities
        raise NotImplementedError('Unknown attr requested')

---

In [95]:
routes_table = pd.read_excel('RouteDetails.xlsx')

In [96]:
routes_table.head()

,RoutePosition,latitude,longitude,depot_latitude,depot_longitude,RouteNumber,DepotCode,DepotName,Depot location Postcode,Postcode,...,AverageBoxes,CustomerId,HouseNumber,Area,total_distance_miles,out_distance_miles,return_distance_miles,drop_distance_miles,is_drop,leg_mileage
0,0,51.459568,-0.031032,51.405371,-0.175852,11,WIM,Wimbledon,CR4 3TD,SE4 1SF,...,5.444444,1425907.0,199,NaN,8.049421,8.049421,0.0,0.000000,1,8.049421
1,2,51.450027,-0.034410,51.405371,-0.175852,11,WIM,Wimbledon,CR4 3TD,SE4 1AJ,...,4.107143,47608.0,175,NaN,0.746852,0.000000,0.0,0.746852,1,0.000000
2,3,51.450027,-0.034410,51.405371,-0.175852,11,WIM,Wimbledon,CR4 3TD,SE4 1AJ,...,15.714286,38167.0,161,NaN,0.000000,0.000000,0.0,0.000000,1,0.000000
3,4,51.452478,-0.033107,51.405371,-0.175852,11,WIM,Wimbledon,CR4 3TD,SE4 1AF,...,3.000000,902997.0,49,NaN,0.197368,0.000000,0.0,0.197368,1,0.000000
4,6,51.452478,-0.033107,51.405371,-0.175852,11,WIM,Wimbledon,CR4 3TD,SE4 1AF,...,1.142857,73768.0,11,NaN,0.000000,0.000000,0.0,0.000000,1,0.000000


In [128]:
routes = routes_table.to_dict('records')

---

In [129]:
N_VEHICLES = 10
VEHICLE_CAPACITY = 50

---

In [130]:
routes_objs = list()
for route_id, paths in groupby(routes, key=itemgetter('RouteNumber')):
    routes_objs.append(Route(paths, n_vehicles=N_VEHICLES, veh_capacity=VEHICLE_CAPACITY))

---

In [131]:
[f'ID: {i}. N_DROPS: {route_obj._n_drops}' for i, route_obj in enumerate(routes_objs)]

['ID: 0. N_DROPS: 131',
 'ID: 1. N_DROPS: 118',
 'ID: 2. N_DROPS: 134',
 'ID: 3. N_DROPS: 111',
 'ID: 4. N_DROPS: 119',
 'ID: 5. N_DROPS: 141',
 'ID: 6. N_DROPS: 141',
 'ID: 7. N_DROPS: 118',
 'ID: 8. N_DROPS: 119',
 'ID: 9. N_DROPS: 132',
 'ID: 10. N_DROPS: 113',
 'ID: 11. N_DROPS: 124',
 'ID: 12. N_DROPS: 107',
 'ID: 13. N_DROPS: 111',
 'ID: 14. N_DROPS: 132',
 'ID: 15. N_DROPS: 125',
 'ID: 16. N_DROPS: 122',
 'ID: 17. N_DROPS: 100',
 'ID: 18. N_DROPS: 113',
 'ID: 19. N_DROPS: 123',
 'ID: 20. N_DROPS: 135',
 'ID: 21. N_DROPS: 117',
 'ID: 22. N_DROPS: 106',
 'ID: 23. N_DROPS: 130',
 'ID: 24. N_DROPS: 139',
 'ID: 25. N_DROPS: 111',
 'ID: 26. N_DROPS: 119',
 'ID: 27. N_DROPS: 110',
 'ID: 28. N_DROPS: 127',
 'ID: 29. N_DROPS: 127',
 'ID: 30. N_DROPS: 123',
 'ID: 31. N_DROPS: 114',
 'ID: 32. N_DROPS: 115',
 'ID: 33. N_DROPS: 109',
 'ID: 34. N_DROPS: 138',
 'ID: 35. N_DROPS: 140',
 'ID: 36. N_DROPS: 122',
 'ID: 37. N_DROPS: 117',
 'ID: 38. N_DROPS: 128',
 'ID: 39. N_DROPS: 128',
 'ID: 40. 

---

In [132]:
# 154 - 18 штук
# 599 - много

In [133]:
ROUTE_ID = 154

---

In [134]:
route_obj = routes_objs[ROUTE_ID]

print(f'Number of Drops: {route_obj._n_drops}')
print(f'Max Distance: {route_obj._dist_mx.max()} (m)')
print(f'Vehicle capacity: {route_obj.vehicle_capacity}')
print(f'Vehicle number: {route_obj._n_vehicles}')
print(f'Demands: {route_obj["demands"]}')

Number of Drops: 18
Max Distance: 20637.0 (m)
Vehicle capacity: 50
Vehicle number: 10
Demands: [0.0, 4.0, 2.0, 5.0, 10.0, 5.0, 3.0, 1.0, 1.0, 3.0, 5.0, 3.0, 6.0, 3.0, 7.0, 1.0, 1.0, 5.0]


In [135]:
solution = route_obj.solve_or()

if solution:
    vehicle_routes, routes_loads, routes_dists = get_routes(route_obj, solution)
else:
    raise RuntimeError('There is no any solution')

In [136]:
vehicle_routes

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 3, 2, 1, 13, 4, 0],
 [0, 5, 16, 10, 9, 8, 7, 15, 12, 17, 14, 11, 6, 0]]

---

In [137]:
MarketIcon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/862/862893.png', icon_size=[40, 40])
DepotIcon = Icon(icon_url='https://granritual.ru/image/catalog/services/save.png', icon_size=[40, 40])

In [138]:
info_box_template = """
<dl>
<dt>Координаты:</dt><dd>{0}</dd>
<dt>Количество коробок:</dt><dd>{1}</dd>
</dl>
"""

route_info_box_template = """
<dl>
<dt>Длина пути:</dt><dd>{0} (m)</dd>
<dt>Суммарная нагрузка:</dt><dd>{1} (boxes)</dd>
</dl>
"""

---

In [139]:
routes_loads

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 24.0, 41.0]

In [140]:
routes_dists

[0, 0, 0, 0, 0, 0, 0, 0, 26543, 47769]

In [141]:
centroid = get_centroid(route_obj._drops)
_map = Map(center=centroid)

depot_marker = Marker(location=route_obj._depot, icon=DepotIcon, draggable=False)
markets_markers = [Marker(location=drop, icon=MarketIcon, draggable=False) for drop in route_obj._drops[1:]]
for i, marker in enumerate(markets_markers, start=1):
    marker.popup = HTML(info_box_template.format(route_obj._drops[i], route_obj._boxes[i]))

for i, vroute in enumerate(vehicle_routes):
    line = Polyline(
        locations=[route_obj._drops[node] for node in vroute],
        color=vroutes_colors[i],
        fill=False
    )
    line.popup = HTML(route_info_box_template.format(routes_dists[i], routes_loads[i]))
    _map.add_layer(line)
    
_map.add_layer(MarkerCluster(
    markers=[depot_marker] + markets_markers
))

_map

Map(center=[51.49111588888888, -0.13472383333333332], controls=(ZoomControl(options=['position', 'zoom_in_text…

---

In [142]:
N_VEHICLES = 10
VEHICLE_CAPACITY = 50

In [143]:
routes_objs = list()
for route_id, paths in groupby(routes, key=itemgetter('RouteNumber')):
    routes_objs.append(Route(paths, n_vehicles=N_VEHICLES, veh_capacity=VEHICLE_CAPACITY))

In [144]:
[f'ID: {i}. N_DROPS: {route_obj._n_drops}' for i, route_obj in enumerate(routes_objs)]

['ID: 0. N_DROPS: 131',
 'ID: 1. N_DROPS: 118',
 'ID: 2. N_DROPS: 134',
 'ID: 3. N_DROPS: 111',
 'ID: 4. N_DROPS: 119',
 'ID: 5. N_DROPS: 141',
 'ID: 6. N_DROPS: 141',
 'ID: 7. N_DROPS: 118',
 'ID: 8. N_DROPS: 119',
 'ID: 9. N_DROPS: 132',
 'ID: 10. N_DROPS: 113',
 'ID: 11. N_DROPS: 124',
 'ID: 12. N_DROPS: 107',
 'ID: 13. N_DROPS: 111',
 'ID: 14. N_DROPS: 132',
 'ID: 15. N_DROPS: 125',
 'ID: 16. N_DROPS: 122',
 'ID: 17. N_DROPS: 100',
 'ID: 18. N_DROPS: 113',
 'ID: 19. N_DROPS: 123',
 'ID: 20. N_DROPS: 135',
 'ID: 21. N_DROPS: 117',
 'ID: 22. N_DROPS: 106',
 'ID: 23. N_DROPS: 130',
 'ID: 24. N_DROPS: 139',
 'ID: 25. N_DROPS: 111',
 'ID: 26. N_DROPS: 119',
 'ID: 27. N_DROPS: 110',
 'ID: 28. N_DROPS: 127',
 'ID: 29. N_DROPS: 127',
 'ID: 30. N_DROPS: 123',
 'ID: 31. N_DROPS: 114',
 'ID: 32. N_DROPS: 115',
 'ID: 33. N_DROPS: 109',
 'ID: 34. N_DROPS: 138',
 'ID: 35. N_DROPS: 140',
 'ID: 36. N_DROPS: 122',
 'ID: 37. N_DROPS: 117',
 'ID: 38. N_DROPS: 128',
 'ID: 39. N_DROPS: 128',
 'ID: 40. 

In [145]:
ROUTE_ID = 297

In [146]:
route_obj = routes_objs[ROUTE_ID]

print(f'Number of Drops: {route_obj._n_drops}')
print(f'Max Distance: {route_obj._dist_mx.max()} (m)')
print(f'Vehicle capacity: {route_obj.vehicle_capacity}')
print(f'Vehicle number: {route_obj._n_vehicles}')
print(f'Demands: {route_obj["demands"]}')

Number of Drops: 87
Max Distance: 67461.0 (m)
Vehicle capacity: 50
Vehicle number: 10
Demands: [0.0, 1.0, 2.0, 1.0, 4.0, 2.0, 3.0, 3.0, 1.0, 5.0, 1.0, 2.0, 8.0, 3.0, 16.0, 2.0, 1.0, 3.0, 7.0, 11.0, 8.0, 2.0, 5.0, 2.0, 3.0, 6.0, 2.0, 2.0, 1.0, 5.0, 5.0, 2.0, 3.0, 4.0, 1.0, 1.0, 3.0, 3.0, 4.0, 5.0, 5.0, 1.0, 4.0, 2.0, 7.0, 2.0, 7.0, 7.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 3.0, 4.0, 1.0, 1.0, 10.0, 3.0, 1.0, 13.0, 9.0, 6.0, 2.0, 1.0, 5.0, 1.0, 4.0, 5.0, 4.0, 6.0, 3.0, 3.0, 7.0, 1.0, 1.0, 1.0, 4.0, 2.0, 4.0, 1.0, 8.0, 1.0, 1.0, 2.0, 4.0]


In [147]:
solution = route_obj.solve_or()

if solution:
    vehicle_routes, routes_loads, routes_dists = get_routes(route_obj, solution)
else:
    raise RuntimeError('There is no any solution')

In [148]:
vehicle_routes

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 2, 4, 5, 3, 13, 21, 25, 41, 38, 39, 40, 24, 23, 22, 34, 48, 49, 57, 0],
 [0, 18, 17, 16, 15, 20, 19, 44, 42, 43, 50, 51, 52, 0],
 [0, 86, 64, 63, 70, 69, 68, 67, 66, 65, 71, 72, 73, 81, 84, 85, 0],
 [0, 6, 8, 7, 10, 1, 14, 11, 12, 9, 0],
 [0, 46, 45, 36, 35, 37, 26, 27, 28, 30, 29, 31, 33, 32, 47, 53, 0],
 [0, 56, 55, 54, 61, 62, 58, 60, 59, 0],
 [0, 77, 76, 75, 74, 78, 80, 79, 82, 83, 0]]

In [149]:
MarketIcon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/862/862893.png', icon_size=[40, 40])
DepotIcon = Icon(icon_url='https://granritual.ru/image/catalog/services/save.png', icon_size=[40, 40])

In [150]:
info_box_template = """
<dl>
<dt>Координаты:</dt><dd>{0}</dd>
<dt>Количество коробок:</dt><dd>{1}</dd>
</dl>
"""

route_info_box_template = """
<dl>
<dt>Длина пути:</dt><dd>{0} (m)</dd>
<dt>Суммарная нагрузка:</dt><dd>{1} (boxes)</dd>
</dl>
"""

In [151]:
centroid = get_centroid(route_obj._drops)
_map = Map(center=centroid)

depot_marker = Marker(location=route_obj._depot, icon=DepotIcon, draggable=False)
markets_markers = [Marker(location=drop, icon=MarketIcon, draggable=False) for drop in route_obj._drops[1:]]
for i, marker in enumerate(markets_markers, start=1):
    marker.popup = HTML(info_box_template.format(route_obj._drops[i], route_obj._boxes[i]))

for i, vroute in enumerate(vehicle_routes):
    line = Polyline(
        locations=[route_obj._drops[node] for node in vroute],
        color=vroutes_colors[i],
        fill=False
    )
    line.popup = HTML(route_info_box_template.format(routes_dists[i], routes_loads[i]))
    _map.add_layer(line)
    
_map.add_layer(MarkerCluster(
    markers=[depot_marker] + markets_markers
))

_map

Map(center=[51.03267055747126, -0.26911948275862096], controls=(ZoomControl(options=['position', 'zoom_in_text…